In [1]:
import torch
torch.cuda.get_device_name(torch.cuda.current_device())

'Tesla V100-SXM2-16GB'

In [20]:
from IPython.display import clear_output
from plotly import offline as plotly
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torch import nn
import plotly.graph_objs as go
import pandas as pd
import torch
import tqdm
import os

plotly.init_notebook_mode(True)
np = pd.np

### Hyper Parameters
LEARNING_RATE = 0.001 # 0.01 and 0.005 worked eventually
BATCH_SIZE = 50000
NUM_EPOCHS = 1
INPUT_SIZE = NUM_FEATURES = 1
HIDDEN_SIZE = 200
NUM_CLASSES = 2
SEQ_LENGTH = 10
NUM_LAYERS = 1

In [3]:
def read_returns(pair):
    path = f's3://mikeokslonger-ticks/returns.parquet/pair={pair}/part.0.parquet'
    df = pd.read_parquet(path, columns=['time', 'relative_returns', 'buy', 'quantity', 'price'])
    return df

def create_features(df):
    if df is None:
        return None
    one_hot = np.array([[1, 0],
                       [0, 1]], dtype='float32')
    return df.assign(buy=lambda x: x.buy.astype(float)) \
        .assign(label=lambda x: (x.shift(-1).relative_returns > 0.).astype(int)) \
        .assign(label_onehot=lambda x: x['label'].apply(lambda i: one_hot[i])) \
        .assign(time=lambda x: (x['time'] - 20180101000000).astype(float))[1:]

def filter_null_returns(df):
    return df[df.relative_returns.abs() > 0.]

pairs = ['ADX-USD', 'AIR-USD', 'AMM-USD', 'ATB-USD', 'ATM-USD', 'B2X-USD', 'BCH-USD', 'BCN-USD', 'BMC-USD', 'BNT-USD', 'BQX-USD', 'BTCA-USD', 'BTC-USD', 'BTG-USD', 'BTM-USD', 'BTX-USD', 'CAT-USD', 'CDT-USD', 'CLD-USD', 'CL-USD', 'CND-USD', 'CTR-USD', 'CVC-USD', 'DASH-USD', 'DATA-USD', 'DCN-USD', 'DGB-USD', 'DIM-USD', 'DOGE-USD', 'EBTCOLD-USD', 'EDO-USD', 'EMGO-USD', 'ENJ-USD', 'EOS-USD', 'ETC-USD', 'ETH-USD', 'ETP-USD', 'EVX-USD', 'FUEL-USD', 'FUN-USD', 'ICOS-USD', 'ICX-USD', 'KMD-USD', 'LOC-USD', 'LSK-USD', 'LTC-USD', 'MAID-USD', 'MANA-USD', 'MCO-USD', 'NEO-USD', 'NGC-USD', 'NXT-USD', 'OAX-USD', 'OMG-USD', 'PLR-USD', 'PPC-USD', 'PRG-USD', 'QTUM-USD', 'SMART-USD', 'SMS-USD', 'SNC-USD', 'SNT-USD', 'STRAT-USD', 'STU-USD', 'STX-USD', 'SUB-USD', 'SUR-USD', 'SWFTC-USD', 'TNT-USD', 'TRX-USD', 'UGT-USD', 'UTT-USD', 'VEN-USD', 'VERI-USD', 'VIB-USD', 'WMGO-USD', 'WRC-USD', 'XDN-USD', 'XEM-USD', 'XMR-USD', 'XTZ-USD', 'XUC-USD', 'XVG-USD', 'ZEC-USD', 'ZRX-USD', 'ZSC-USD']
pairs = pairs
df = pd.concat([read_returns(pair).pipe(filter_null_returns).pipe(create_features) for pair in tqdm.tqdm(pairs)]) # Full dataset

100%|██████████| 86/86 [00:58<00:00,  1.47it/s]


In [12]:
class Ticks(Dataset):
    def __init__(self, data, batch_size):
        self.x_data = data[:, 1:2].copy().astype('float32')
        self.y_data = data[:, -1].copy()#.astype('int')
        self.len = len(self.x_data) - SEQ_LENGTH - ((len(self.x_data) - SEQ_LENGTH) % batch_size)
    
    def __getitem__(self, index):
        x = self.x_data[index: index + SEQ_LENGTH]
        return x, self.y_data[index + SEQ_LENGTH - 1]
    
    def __len__(self):
        return self.len


test_validation_split = int(len(df) * 0.8)
dataset_train = Ticks(df[['time', 'relative_returns', 'buy', 'quantity', 'label', 'label_onehot']].values[:test_validation_split], BATCH_SIZE)
dataset_test = Ticks(df[['time', 'relative_returns', 'buy', 'quantity', 'label', 'label_onehot']].values[test_validation_split:], BATCH_SIZE)
dataloader_train = DataLoader(dataset=dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
dataloader_test = DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [14]:
### Create Model

class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = SEQ_LENGTH
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        ### (LAYERS, BATCH_SIZE, HIDDEN_SIZE)
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).cuda() # Hidden
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).cuda() # Cell state
        _, (h_out, _) = self.lstm(x, (h0, c0))
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        activated = self.sig(out)
        return activated


losses = []
accuracies = []
test_accuracies = []
test_predicted_positives = []

# TRAIN

lstm = LSTM(NUM_CLASSES, INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS).cuda()
#lstm.load_state_dict(torch.load('models/model134.pt'))

In [15]:
class BCEUniformLoss(torch.nn.BCELoss):
    def forward(self, input, target):
        normal_result = super(BCEUniformLoss, self).forward(input, target)
        skew = torch.abs(input.data[:, [1]].sum() - trainY[:, [1]].sum())
        adjustment_factor = (skew / trainY[:, [1]].sum()) ** 2
        return normal_result + normal_result * adjustment_factor

In [16]:
print(len(dataset_train))
print(len(dataset_test))

26150000
6500000


In [21]:
#loss_fn = torch.nn.MSELoss()
#loss_fn = torch.nn.CrossEntropyLoss()
#loss_fn = torch.nn.BCELoss()
loss_fn = BCEUniformLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=LEARNING_RATE)

In [22]:
for epoch in list(range(50, 75)):
    correct = 0
    num_positive_returns = 0
    predicted_positive_returns = 0
    num_observations = 0
    for i, (trainX, trainY) in tqdm.tqdm(enumerate(dataloader_train), total=len(dataset_train) // BATCH_SIZE):
        trainX, trainY = Variable(trainX).cuda(), Variable(trainY).cuda()
        outputs = lstm(trainX)
        correct += torch.max(outputs.data, 1)[1].eq(torch.max(trainY.data, 1)[1]).sum()
        optimizer.zero_grad()
        loss = loss_fn(outputs, trainY)
        loss.backward()
        optimizer.step()
        predicted_positive_returns += torch.max(outputs.data, 1)[1].sum()
        num_positive_returns += trainY[:, [1]].sum()
        num_observations += len(trainY)

    accuracies.append(correct / len(dataloader_train.dataset))
    losses.append(loss.cpu().data[0])
    num_positive_returns = num_positive_returns.cpu().data[0]

    clear_output(True)
    print(f'epoch {epoch + 1}/{NUM_EPOCHS}')
    print(f'actual positive returns {num_positive_returns}/{num_observations}: {num_positive_returns/num_observations}')
    print(f'predicted positive_returns {predicted_positive_returns}/{num_observations}: {predicted_positive_returns/num_observations}')
    
    correct = 0
    num_positive_returns = 0
    predicted_positive_returns = 0
    num_observations = 0
    for i, (testX, testY) in tqdm.tqdm(enumerate(dataloader_test), total=len(dataset_test) // BATCH_SIZE):
        testX, testY = Variable(testX).cuda(), Variable(testY).cuda()
        outputs = lstm(testX)
        predicted_labels = torch.max(outputs.data, 1)[1]
        correct += predicted_labels.cpu().eq(testY.cpu().data[:, [1]].view(-1).long()).sum()
        predicted_positive_returns += torch.max(outputs.data, 1)[1].sum()
        num_positive_returns += testY[:, [1]].sum()

    loss = loss_fn(outputs, trainY)
    accuracy = correct / len(dataloader_test.dataset)
    test_accuracies.append(accuracy)
    num_positive_returns = num_positive_returns.cpu().data[0]
    test_predicted_positives.append(predicted_positive_returns/len(dataloader_test.dataset))
    
    print(f'loss: {loss.data[0]}, accuracy: {accuracy}')
    print(f'actual positive returns {num_positive_returns}/{len(dataloader_test.dataset)}: {num_positive_returns/len(dataloader_test.dataset)}')
    print(f'predicted positive_returns {predicted_positive_returns}/{len(dataloader_test.dataset)}: {predicted_positive_returns/len(dataloader_test.dataset)}')
    
    plotly.iplot(go.Figure(data=[go.Scatter(x=list(range(len(losses))), y=losses, name='loss'),
                                 go.Scatter(x=list(range(len(accuracies))), y=accuracies, name='train_accuracy', yaxis='y2'),
                                 go.Scatter(x=list(range(len(test_accuracies))), y=test_accuracies, name='test_accuracy', yaxis='y2'),
                                 go.Scatter(x=list(range(len(test_predicted_positives))), y=test_predicted_positives, name='test_predicted_positives', yaxis='y2')],
                           layout=go.Layout(yaxis={'title': 'loss'}, yaxis2={'title': 'accuracy', 'overlaying': 'y', 'side': 'right'})))
    _ = plotly.plot(go.Figure(data=[go.Scatter(x=list(range(len(losses))), y=losses, name='loss'),
                                    go.Scatter(x=list(range(len(accuracies))), y=accuracies, name='train_accuracy', yaxis='y2'),
                                    go.Scatter(x=list(range(len(test_accuracies))), y=test_accuracies, name='test_accuracy', yaxis='y2'),
                                    go.Scatter(x=list(range(len(test_predicted_positives))), y=test_predicted_positives, name='test_predicted_positives', yaxis='y2')],
                              layout=go.Layout(yaxis={'title': 'loss'}, yaxis2={'title': 'accuracy', 'overlaying': 'y', 'side': 'right'})),
               filename=f'models/training/training{epoch}.html')
    
    df = pd.DataFrame(outputs.data.cpu().numpy()).assign(p1=lambda x: x[1] / (x[0] + x[1])).assign(y=1).sample(n=500)
    plotly.iplot(go.Figure(data=[go.Scatter(x=df.p1, y=df.y, mode='markers')],
                           layout=go.Layout(xaxis=dict(range=[0, 1]), yaxis=dict(range=[0, 2]))))
    _ = plotly.plot(go.Figure(data=[go.Scatter(x=df.p1, y=df.y, mode='markers')],
                              layout=go.Layout(xaxis=dict(range=[0, 1]), yaxis=dict(range=[0, 2]))),
                    filename=f'models/predictions/scatter{epoch}.html')

    torch.save(lstm.state_dict(), f'models/model{epoch}.pt')


epoch 75/1
actual positive returns 13070494.0/26150000: 0.49982768642447417
predicted positive_returns 13096878/26150000: 0.5008366347992352


100%|██████████| 130/130 [00:35<00:00,  3.66it/s]

loss: 0.8079915046691895, accuracy: 0.6850918461538461
actual positive returns 3249802.0/6500000: 0.49996953846153847
predicted positive_returns 3262737/6500000: 0.5019595384615385
